In [ ]:
import re

In [ ]:
classpath_file_subpath = ".classpath"
dependency_declaration_subpath = "META-INF/MANIFEST.MF"
dependency_declaration_keyword = "Require-Bundle"
dependency_declaration_word_end = ":"

class Dependency():
    def __init__(self, dependency_manifest_string):
        params = dependency_manifest_string.split(';')
        self.name = params[0]
        self.exported = dependency_manifest_string.find("reexport") > 0
    def __str__(self):
        return f"{self.name} : exported {self.exported}"
    def __repr__(self):
        return f"{self.name} : exported {self.exported}"
import functools
def read_pattern(lines): 
    require_block_declaration_pattern = re.compile(r"({})*:[\ ]*.*".format(dependency_declaration_keyword))
    block_declaration_pattern = re.compile(r"[a-zA-Z-]*:[\ ]*.*") 
    first_match = False
    first_index = 0
    last_index = 0
    for index, i in enumerate(lines):
        if re.fullmatch(require_block_declaration_pattern, i) is not None:
            first_index = index
            first_match = True
            continue
        if first_match and re.fullmatch(block_declaration_pattern, i) is not None:
            lines = lines[first_index:index]
            break
    lines = list(map(lambda x: x.strip(), lines))
    lines = ''.join(lines)
    lines = lines.strip()[lines.find(dependency_declaration_keyword) + len(dependency_declaration_keyword + dependency_declaration_word_end):]
    def split_by_commas(lines): 
        parenthesis_stack = []
        new_lines = []
        last_index = 0
        for index, i in enumerate(lines):
            if (i == '"'):
                if (len(parenthesis_stack) > 0):
                    parenthesis_stack.pop()
                else:
                    parenthesis_stack.append(i)
            elif (i == ','):
                if len(parenthesis_stack) == 0:
                    new_lines.append(lines[last_index:index])
                    last_index = index + 1
        new_lines.append(lines[last_index:])
        return new_lines
    lines = split_by_commas(lines)
    # can be case when ',' was inside brackets
    print(lines)
    dependencies = list(map(lambda x: Dependency(x), lines))
    print(dependencies)
    return dependencies


test = '''
Manifest-Version: 1.0
Automatic-Module-Name: org.eclipse.xtext
Bundle-SymbolicName: org.eclipse.xtext;singleton:=true
Require-Bundle: org.eclipse.emf.ecore.xmi;bundle-version="2.16.0";visi
 bility:=reexport,org.eclipse.emf.ecore;bundle-version="2.20.0";visibi
 lity:=reexport,org.eclipse.emf.common;bundle-version="2.17.0",org.ant
 lr.runtime;bundle-version="[3.2.0,3.2.1)";visibility:=reexport,com.go
 ogle.inject;bundle-version="3.0.0";visibility:=reexport
Bundle-ManifestVersion: 2
Bundle-RequiredExecutionEnvironment: JavaSE-1.8
Bundle-ActivationPolicy: lazy
Eclipse-SourceReferences: scm:git:https://github.com/eclipse/xtext-cor
 e.git;path="org.eclipse.xtext";commitId=c9b37ed42cd453ac77772f4b24ce4
 45df60b4cc8
Bundle-Vendor: Eclipse Xtext
Import-Package: org.apache.log4j;version="1.2.15",org.osgi.framework
Export-Package: org.eclipse.xtext,org.eclipse.xtext.build;x-friends:="
 org.eclipse.xtext.testing,  org.eclipse.xtext.ide,  org.eclipse.xtext
 .junit4,  org.eclipse.xtext.tests,  org.eclipse.xtend.core.tests",org
 .eclipse.xtext.common,org.eclipse.xtext.common.parser.packrat;x-frien
 ds:="org.eclipse.xtext.generator",org.eclipse.xtext.common.parser.pac
 krat.consumers;x-friends:="org.eclipse.xtext.generator",org.eclipse.x
 text.common.services,org.eclipse.xtext.conversion,org.eclipse.xtext.c
 onversion.impl,org.eclipse.xtext.debug;x-friends:="org.eclipse.xtext.
 builder,
'''
read_pattern(test.splitlines())

In [1]:
from dependency import *
bundle = Bundle(Path("/home/algor/.p2/"), "org.eclipse.xtext")
bundle.update_jars()
bundle.jars
bundle.get_manifest_file_lines()
bundle.update_dependencies()
bundle.dependencies

[ org.eclipse.emf.ecore.xmi : exported True,
 org.eclipse.emf.ecore : exported True,
 org.eclipse.emf.common : exported False,
 org.antlr.runtime : exported True,
 com.google.inject : exported True,
 org.objectweb.asm : exported False,
 org.eclipse.emf.mwe.core : exported True,
 org.eclipse.emf.mwe.utils : exported True,
 org.eclipse.xtend : exported False,
 org.eclipse.xtend.typesystem.emf : exported False,
 org.eclipse.xtext.util : exported True,
 org.eclipse.core.runtime : exported False,
 org.eclipse.xtend.lib : exported False,
 org.eclipse.equinox.common : exported False]

In [4]:
test = """
Manifest-Version: 1.0
Automatic-Module-Name: org.eclipse.xtext
Bundle-SymbolicName: org.eclipse.xtext;singleton:=true
Require-Bundle: org.eclipse.emf.ecore.xmi;bundle-version="2.16.0";visi
 bility:=reexport,org.eclipse.emf.ecore;bundle-version="2.20.0";visibi
 lity:=reexport,org.eclipse.emf.common;bundle-version="2.17.0",org.ant
 lr.runtime;bundle-version="[3.2.0,3.2.1)";visibility:=reexport,com.go
 ogle.inject;bundle-version="3.0.0";visibility:=reexport,org.objectweb
 .asm;bundle-version="[8.0.1,8.1.0)";resolution:=optional,org.eclipse.
 emf.mwe.core;bundle-version="1.3.21";resolution:=optional;visibility:
 =reexport,org.eclipse.emf.mwe.utils;bundle-version="1.3.21";resolutio
 n:=optional;visibility:=reexport,org.eclipse.xtend;bundle-version="2.
 2.0";resolution:=optional,org.eclipse.xtend.typesystem.emf;bundle-ver
 sion="2.2.0";resolution:=optional,org.eclipse.xtext.util;visibility:=
 reexport,org.eclipse.core.runtime;bundle-version="3.13.0";resolution:
 =optional;x-installation:=greedy,org.eclipse.xtend.lib;resolution:=op
 tional,org.eclipse.equinox.common;bundle-version="3.9.0"
Bundle-ManifestVersion: 2
Bundle-RequiredExecutionEnvironment: JavaSE-1.8
Bundle-ActivationPolicy: lazy
Eclipse-SourceReferences: scm:git:https://github.com/eclipse/xtext-cor
 e.git;path="org.eclipse.xtext";commitId=c9b37ed42cd453ac77772f4b24ce4
 45df60b4cc8
Bundle-Vendor: Eclipse Xtext
Import-Package: org.apache.log4j;version="1.2.15",org.osgi.framework
Export-Package: org.eclipse.xtext,org.eclipse.xtext.build;x-friends:="
 org.eclipse.xtext.testing,  org.eclipse.xtext.ide,  org.eclipse.xtext
 .junit4,  org.eclipse.xtext.tests,  org.eclipse.xtend.core.tests",org
 .eclipse.xtext.common,org.eclipse.xtext.common.parser.packrat;x-frien
 ds:="org.eclipse.xtext.generator",org.eclipse.xtext.common.parser.pac
 krat.consumers;x-friends:="org.eclipse.xtext.generator",org.eclipse.x
 text.common.services,org.eclipse.xtext.conversion,org.eclipse.xtext.c
 onversion.impl,org.eclipse.xtext.debug;x-friends:="org.eclipse.xtext.
 builder,  org.eclipse.xtext.ide.tests,  org.eclipse.xtext.xbase",org.
 eclipse.xtext.diagnostics,org.eclipse.xtext.documentation,org.eclipse
 .xtext.documentat
"""
test=test.split("\n")

In [5]:
parse_manifest_file_lines(test)


Manifest-Version: 1.0
Automatic-Module-Name: org.eclipse.xtext
Bundle-SymbolicName: org.eclipse.xtext;singleton:=true
Require-Bundle: org.eclipse.emf.ecore.xmi;bundle-version="2.16.0";visi
 bility:=reexport,org.eclipse.emf.ecore;bundle-version="2.20.0";visibi
 lity:=reexport,org.eclipse.emf.common;bundle-version="2.17.0",org.ant
 lr.runtime;bundle-version="[3.2.0,3.2.1)";visibility:=reexport,com.go
 ogle.inject;bundle-version="3.0.0";visibility:=reexport,org.objectweb
 .asm;bundle-version="[8.0.1,8.1.0)";resolution:=optional,org.eclipse.
 emf.mwe.core;bundle-version="1.3.21";resolution:=optional;visibility:
 =reexport,org.eclipse.emf.mwe.utils;bundle-version="1.3.21";resolutio
 n:=optional;visibility:=reexport,org.eclipse.xtend;bundle-version="2.
 2.0";resolution:=optional,org.eclipse.xtend.typesystem.emf;bundle-ver
 sion="2.2.0";resolution:=optional,org.eclipse.xtext.util;visibility:=
 reexport,org.eclipse.core.runtime;bundle-version="3.13.0";resolution:
 =optional;x-installation:=gr

[ org.eclipse.emf.ecore.xmi : exported True,
 org.eclipse.emf.ecore : exported True,
 org.eclipse.emf.common : exported False,
 org.antlr.runtime : exported True,
 com.google.inject : exported True,
 org.objectweb.asm : exported False,
 org.eclipse.emf.mwe.core : exported True,
 org.eclipse.emf.mwe.utils : exported True,
 org.eclipse.xtend : exported False,
 org.eclipse.xtend.typesystem.emf : exported False,
 org.eclipse.xtext.util : exported True,
 org.eclipse.core.runtime : exported False,
 org.eclipse.xtend.lib : exported False,
 org.eclipse.equinox.common : exported False]